In [12]:
# read midi and query data
# insert it into dataframe

import os
import pandas as pd

folder_data = "Data query dan midi"

# test query
dir_query = os.path.join(folder_data, "Test query MIR-QBSH")
# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH")

# all query
# dir_query = os.path.join(folder_data, "Query_MIR_QBSH")
# dir_query = os.path.join(folder_data, "Query_IOACAS_QBH")

# all midi
dir_midi = os.path.join(folder_data, "Database midi MIR-QBSH")
# dir_midi = os.path.join(folder_data, "Database midi IOACAS-QBH")


dir_query_DNN = os.path.join(dir_query, "DNN-LSTM")

# to be used as index in inverted index database
list_dir_midi = []

list_query_name = []
list_query_DNN_name = []

df_query = []
df_midi = []
df_query_DNN = []

truth = []
truth_DNN = []

# read and insert semitone from query to list of dataframe
for file in os.listdir(dir_query):
    if file.endswith(".csv"):
        list_query_name.append(file.replace(".csv",""))
        file_path = os.path.join(dir_query, file)
        fields = ["semitone"]
        temp_df = pd.read_csv(file_path, usecols=fields)
        df_query.append(temp_df.to_numpy().flatten())
        truth.append(file.split("-")[2])
    elif file.endswith(".pv"):
        list_query_name.append(file.replace(".pv",""))
        file_path = os.path.join(dir_query, file)
        temp_df = pd.read_csv(file_path)
        temp_df = temp_df.to_numpy().flatten()
        round_df = []
        for i in temp_df:
            if i!=0:
                round_df.append(round(i))
        df_query.append(round_df)
        truth.append(file.split("-")[2])

# read and insert note_index or semitone from midi to list of dataframe
for file in os.listdir(dir_midi):
    if file.endswith(".csv"):
        list_dir_midi.append(file.replace(".mid.csv",""))
        file_path = os.path.join(dir_midi, file)
        fields = ["note_index"]
        temp_df = pd.read_csv(file_path, usecols=fields)
        df_midi.append(temp_df.to_numpy().flatten())

# read f0 from extracted melody (DNN-LSTM) and convert it to semitone then insert to list of dataframe
for file in os.listdir(dir_query_DNN):
    if file.endswith(".csv"):
        list_query_DNN_name.append(file.replace(".csv",""))
        file_path = os.path.join(dir_query_DNN, file)
        fields = ["semitone"]
        temp_df = pd.read_csv(file_path, usecols=fields)
        df_query_DNN.append(temp_df.to_numpy().flatten())
        truth_DNN.append(file.split("-")[2])

In [2]:
# calculate relative distance
def calc_rel_dis(df):
    res = []
    length = len(df)
    for i in range(length-1):
        dis = float(df[i+1] - df[i])
        res.append(dis)
    res = remove_consecutive(res)
    return res

# remove consecutive distance in list of relative distance 
def remove_consecutive(list):
    i = 0
    while i < len(list)-1:
        if list[i] == list[i+1]:
            del list[i]
        else:
            i = i+1
    return list

# calculate relative distance in all query
dis_query = []
for query in df_query:
    dis_query.append(calc_rel_dis(query))

# calculate relative distance in all midi
dis_midi = []
for midi in df_midi:
    dis_midi.append(calc_rel_dis(midi))

# calculate relative distance in query from melody extraction DNN-LSTM
dis_query_DNN = []
for query in df_query_DNN:
    dis_query_DNN.append(calc_rel_dis(query))

# print(dis_query)
# print(dis_midi)
# print(dis_query_DNN)

In [3]:
# create inverted index for 2-grams, 3-grams, 4-grams in all midi

import hashedindex
index = hashedindex.HashedIndex()

# example:
# midi relative distance : +1 +1 +4
# 2-grams are +1 and +4
# 3-grams are +1 +1 and +1 +3
# 4-grams is +1 +1 +4

# print(dis_midi[0])

# inserting 2-grams as inverted index
for midiNumber in range(len(dis_midi)):
    for note in dis_midi[midiNumber]:
        index.add_term_occurrence(note, list_dir_midi[midiNumber])

# test 2-grams
# index.get_documents((2.0))

# inserting 3-grams as inverted index
for midiNumber in range(len(dis_midi)):
    for noteNumber in range(len(dis_midi[midiNumber])-1):
        term = (dis_midi[midiNumber][noteNumber], dis_midi[midiNumber][noteNumber+1])
        index.add_term_occurrence(term, list_dir_midi[midiNumber])

# test 3-grams
# index.get_documents((2.0, 0.0))

# inserting 4-grams as inverted index
for midiNumber in range(len(dis_midi)):
    for noteNumber in range(len(dis_midi[midiNumber])-2):
        term = (dis_midi[midiNumber][noteNumber], dis_midi[midiNumber][noteNumber+1], dis_midi[midiNumber][noteNumber+2])
        index.add_term_occurrence(term, list_dir_midi[midiNumber])

# test 4-grams
# index.get_documents((2.0, 0.0, -2.0))

In [20]:
import edit_distance

# ref = dis_midi
# hyp = dis_query

counter = 0
list_ratio = []

def get_all_rank(ls):
    res = []
    for i in ls:
        temp = ls.index(max(ls))
        res.append(list_dir_midi[temp])
        ls[temp] = 0
    return res

def get_rank(rank, truth):
    return rank.index(truth)+1

def get_top_ten(rank):
    return rank[:10]

# test match with query data extracted with praat-parselmouth
for i in range(len(dis_query)):
    hyp = dis_query[i]
    list_ratio = []
    for ref in dis_midi:
        sm = edit_distance.SequenceMatcher(a=ref, b=hyp)
        list_ratio.append(sm.ratio())
    rank = get_all_rank(list_ratio)
    rankTruth = get_rank(rank, truth[i])
    topTen = get_top_ten(rank)
    # print("query",list_query_name[i],"top ten is",topTen)
    print("query",list_query_name[i],"truth is on rank",rankTruth)

# test match with query data extracted with DNN-LSTM
for i in range(len(dis_query_DNN)):
    hyp = dis_query_DNN[i]
    list_ratio = []
    for ref in dis_midi:
        sm = edit_distance.SequenceMatcher(a=ref, b=hyp)
        list_ratio.append(sm.ratio())
    rank = get_all_rank(list_ratio)
    # print(rank)
    rankTruth = get_rank(rank, truth_DNN[i])
    topTen = get_top_ten(rank)
    # print("query",list_query_DNN_name[i],"top ten is",topTen)
    print("query",list_query_DNN_name[i],"truth is on rank",rankTruth)

# print(list_ratio)
# print(max(list_ratio))
# print(list_ratio.index(max(list_ratio)))

query year2003-person00001-00013-manual truth is on rank 15
query year2003-person00001-00013-parselmouth truth is on rank 21
query year2003-person00001-00014-manual truth is on rank 1
query year2003-person00001-00014-parselmouth truth is on rank 5
query year2003-person00001-00013-DNN_LSTM truth is on rank 16
query year2003-person00001-00014-DNN_LSTM truth is on rank 6
